In [2]:
# https://www.kaggle.com/c/dog-breed-identification
import numpy as np
import pandas as pd
import keras
import functools
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
import cv2

data_dir = 'data/'
from subprocess import check_output
print(check_output(["ls", data_dir]).decode("utf8"))

labels.csv
sample_submission.csv
submission_inception.csv
test
train
train_inception.json
validation_inception.json



First we will read in the csv's so we can see some more information on the filenames and breeds

In [5]:
df_train = pd.read_csv(data_dir + 'labels.csv')
df_test = pd.read_csv(data_dir + 'sample_submission.csv')

In [6]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [7]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot.head(3)

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
one_hot_labels = np.asarray(one_hot)

In [11]:
im_size = 224

In [12]:
x_train = []
y_train = []
x_test = []

In [13]:
def process_image(image):
    image = img_to_array(image)
    image = imagenet_utils.preprocess_input(image)
    #image = np.expand_dims(image, axis=0)
    return image

def read_process(train_or_test, size, pid):
    img = load_img(os.path.join(data_dir, train_or_test, '{}.jpg').format(pid), target_size=size)
    return process_image(img)
                         
size = (im_size, im_size)
train_image_func = functools.partial(read_process, 'train', size)
test_image_func = functools.partial(read_process, 'test', size)

In [14]:
x_train = np.asarray([train_image_func(pid) for pid in df_train['id']])
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

Train Images shape: (10222, 224, 224, 3) size: 1,538,697,216


In [15]:
x_test = np.asarray([test_image_func(pid) for pid in df_test['id']])

In [16]:
y_train = one_hot.as_matrix()

In [17]:
num_class = y_train.shape[1]
num_class

120

Do a stratified shuffle split for train and validation set. This ensures that each class is present in both sets.

In [18]:
ss = StratifiedShuffleSplit(n_splits=1, train_size=0.7, test_size=0.3)
train_index, valid_index = next(ss.split(np.zeros(len(df_train)), df_train['breed']))

X_train = x_train[train_index]
Y_train = y_train[train_index]
X_valid = x_train[valid_index]
Y_valid = y_train[valid_index]
len(X_train), len(X_valid)

(7155, 3067)

Build the CNN architecture using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview).

We then remove the final layer and replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [19]:
# Create the base pre-trained model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = Flatten()(base_model.output)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=2, verbose=1)]
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
model.fit(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid), callbacks=callbacks, verbose=1)
#model.fit(x_train, y_train, epochs=10, validation_split=0.3, verbose=1)

Train on 7155 samples, validate on 3067 samples
Epoch 1/5
  32/7155 [..............................] - ETA: 2:01:59 - loss: 15.6140 - acc: 0.0312

# Next we will make our predictions.

In [ ]:
preds = model.predict(x_test, verbose=0)

In [ ]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
#sub.insert(0, 'id', df_test[:10]['id'])
#sub.head(5)

sub.iloc[0].max()